In [2]:
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time
import ta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data.get_data import *

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data = get_data('SOL', interval='1h')
#data = get_multi_data(interval = "1h", table = 'close')

In [2]:
import pandas as pd
import numpy as np
import ta
from arch import arch_model

from indicator import *
from utils import *

start = "2023"
end = "2023"

data = get_data("BTC", "1d")
data['avg'] = (data.close + data.low + data.high + data.open)/4
data = data.loc[start : end]

ret = data.close.pct_change().dropna()

ma_1 = 3
ma_2 = 7
ma_3 = 14

atr_ = 24
rsi_ = 14

window = 10

data['returns'] = data.close.pct_change()

data['ma_1'] = ma(data, ma_1)
data['ma_2'] = ma(data, ma_2)
data['ma_3'] = ma(data, ma_3)

data['vol_std'] = ret.rolling(window).std()
data['vol'] = ret.ewm(span = 5).std()

data['sar'], data['sar_down'], data['sar_up'] = sar(data)


data['r_target'] = data['returns'].rolling(3).mean()
data['target'] = np.where(data['r_target'] > 0, 1, 0)



In [3]:
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.8, 0.2], vertical_spacing = 0.01)
#shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]
fig.add_trace(
    go.Candlestick(
        x = data.index , open = data.open, close = data.close,
        high = data.high, low = data.low, name = data.symbol[0],
        yaxis='y1'
    ),
    col = 1, row = 1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['avg'],
        name = 'avg',
        yaxis='y1'
    ),
    row = 1 , col =1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['ma_1'],
        name = 'ma 1',
        marker_color = 'blue',
        yaxis='y1'
    ),
    row = 1 , col =1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['ma_2'],
        name = 'ma 2',
        marker_color = 'orange',
        yaxis='y1'
    ),
    row = 1 , col =1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['ma_3'],
        name = 'ma 3',
        marker_color = 'black',
        yaxis='y1'
    ),
    row = 1 , col =1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['sar_up'],
        name = 'sar up',
        mode = "markers",
        marker_color = 'blue',
        yaxis='y1'
    ),
    row = 1 , col =1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['target'],
        name = 'target',
        marker_color = 'blue',
    ),
    row = 2 , col = 1
)

fig.add_trace(
    go.Scatter(
        x = data.index , y = data['sar_down'],
        name = 'sar down',
        mode = "markers",
        marker_color = 'black',
        yaxis='y1'
    ),
    row = 1 , col =1
)

fig.add_trace(
    go.Scatter(
        x = data.index,
        y = data.vol,
        name = 'vol'
    ),
    row = 2 , col = 1
)

fig.update_xaxes(rangeslider_visible = False , col =1, row =1)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

#Fig = go.FigureWidget(fig)
fig.update_layout(height = 800 , width =1500,
                  margin=dict(
                        l=5, r=0, b=10, t=30,
                        pad=1
                    ),
                  showlegend=True,
                  )
fig.show()

In [4]:
gg

NameError: name 'gg' is not defined

In [ ]:
px.line(data['close'] / data['vol'])

In [ ]:
import seaborn as sns

sns.displot(D)

In [ ]:
# My macd personaliser
D = data['ma_meduim'] - data['t_f']
print(D.mean())

colors = np.where(D < 0, 'red', 'green')
Fig.add_trace(
    go.Bar
    (
        x = D.index ,y = D,
        name="histogramme",
        marker_color = colors
    ),
    row = 2, col =1
)
#Fig.add_hline(D.mean(), col = 1, row = 2)

In [ ]:
# MACD
data['macd'], data['macd_diff'], data['signal'] = macd(data, slow = high, fast = slow, signal = 9)

Fig.add_trace(
    go.Scatter(
        x = data.index, y = data.macd ,
        name = "macd"
    ),
    row = 2, col = 1
)

Fig.add_trace(
    go.Scatter(
        x = data.index ,y = data.macd_signal,
        name="signal"
    ),
    row = 2, col =1
)

colors = np.where(data['macd_diff'] < 0, 'red', 'green')
Fig.add_trace(
    go.Bar
    (
        x = data.index ,y = data.macd_diff,
        name="histogramme",
        marker_color = colors
    ),
    row = 2, col =1
)